## RELEVANCY EVALUATOR

In [1]:
import nest_asyncio 
nest_asyncio.apply()

In [2]:
import sys
import pandas as pd

In [11]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator,FaithfulnessEvaluator,EvaluationResult
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI

In [5]:
### Loading the data from the directory data
documents = SimpleDirectoryReader("./data").load_data()

In [6]:
# Use a DatasetGenerator to build train_dataset and test_dataset
data_generator = DatasetGenerator.from_documents(documents,num_questions_per_chunk=25)

c:\Users\Prince\Desktop\Projects\llama\myenv\Lib\site-packages\llama_index\core\evaluation\dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [7]:
## Generate questions
eval_questions = data_generator.generate_questions_from_nodes()

c:\Users\Prince\Desktop\Projects\llama\myenv\Lib\site-packages\llama_index\core\evaluation\dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [8]:
### These questions would be considered as our train set
eval_questions

['What is the main advantage of fast and accurate algorithms for object detection?',
 'How do current detection systems typically work?',
 'What is the sliding window approach used by systems like models for deformable parts (DPM)?',
 'How does the R-CNN approach differ from the sliding window approach?',
 'What is the purpose of post-processing in object detection systems?',
 'Why are complex pipelines for object detection difficult to optimize?',
 'How is object detection re-formulated in the YOLO system?',
 'What are the advantages of YOLO compared to traditional methods for object detection?',
 'How fast is the base network of YOLO able to run during testing?',
 'What is the average accuracy of YOLO compared to other real-time systems?',
 'How does YOLO differ from techniques based on sliding windows and region proposals in terms of reasoning about the image?',
 'What is the significance of considering detection as a regression problem in YOLO?',
 'How does YOLO optimize direct det

In [9]:
# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
evaluator_gpt4 = RelevancyEvaluator(llm=gpt4)
# create vector index
vector_index = VectorStoreIndex.from_documents(documents)


In [12]:
# define jupyter display function
def display_eval_df(
    query:str, response:Response, eval_result:EvaluationResult
) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

### EVALUATE RESPONSE FOR THE TRAIN SET

In [13]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

In [14]:
display_eval_df(eval_questions[1], response_vector, eval_result)

,Query,Response,Source,Evaluation Result,Reasoning
0,How do current detection systems typically work?,"Current detection systems typically reuse classifications to perform detection. They use a classification for an object and evaluate it at different locations and scales in a test image. Systems like models for deformable parts (DPM) use a sliding window approach, executing the classifier at evenly distributed locations across the entire image. More recent approaches, such as R-CNN, use a region-proposal method to generate potential limitations in an image and then execute a classifier on these pre-set frames. Post-processing is used after classification to refine surrounding frames, eliminate double detections, and score the frames again based on other objects.","People look at an image and immediately know what objects are in the image, where they are and how they interact with each other. The human visual system is fast and accurate, so that we can perform complex tasks such as driving a car with little conscious thinking. Fast, accurate algorithms for object detection would enable computers to drive cars without specialized sensors, to provide real-time, sane information to human users and to unlock the potential for general, responsive robot systems. The current detection systems reuse classifications to perform detection. To detect an object, these systems use a classification for that object and evaluate this at different locations and scales in a test image. Systems such as models for deformable parts, DPM, use a sliding window approach, where the classifier is executed at evenly distributed locations across the entire image. More recent approaches, such as R-CNN, use a regio-proposal method to first generate potential limitations in an ...",Pass,YES


## FAITHFULNESS EVALUATOR

In [15]:
llm = OpenAI(model="gpt-4", temperature=0.0)
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(eval_questions[1])
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True
